In [2]:
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import time
import csv
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

In [3]:
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    #認証情報設定
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    #アクセス
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).get_worksheet(0)
    return worksheet

# jsonfile
jsonf = "ikkyosai-order-046c29b43754.json"
# スプレットシートキー指定
spread_sheet_key = "1UrCkBZWG0SGWIhxbhBUYVIoxOA_x49UqQ-6oGV3JRmg"
ws = connect_gspread(jsonf,spread_sheet_key)


In [4]:
df=pd.read_csv("output.csv",encoding="shift-jis")

In [5]:
def reload():
  with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(ws.get_all_values())
  df=pd.read_csv("output.csv",encoding="shift-jis")
reload()

In [6]:
def money_count():
  reload()
  df.fillna("a")
  reload()
  for i in range(len(df)):
    if df.iloc[i,4]=="a" or df.iloc[i,4]==0:
      time.sleep(1)
      pay=0
      order=df.iloc[i,2]
      order_list=order.split(", ")
      for divide_order in order_list:
        if divide_order=="飲み物のみ":
            pay=200
        elif "セット" in divide_order:
            pay+=500
        else:
            pay+=300
      ws.update_cell(i+2,5,pay)
  def show_reload_done():
    popup = tk.Toplevel()
    popup.title("更新完了")
    
    # ラベルの作成
    label = ttk.Label(popup, text="更新が完了しました。", background='#7fffd4', foreground='#333333')
    label.pack(padx=20, pady=20)
    
    # OKボタンの作成
    ok_button = ttk.Button(popup, text="OK", command=popup.destroy)
    ok_button.pack(pady=10)
    
    # ポップアップの表示
    popup.mainloop()
  reload()
  show_reload_done()

In [7]:
reload()
df.head()

,タイムスタンプ,座席番号を入力してください！,注文内容を選んでください、セット、飲み物のみの場合は次の質問にて飲み物を選んでください！,飲み物を選んでください！,Unnamed: 4,Unnamed: 5
0,2024/06/28 13:35:13,21,ケーキ１セット,ホットコーヒー,NaN,会計済
1,2024/06/28 13:37:19,8,ケーキ３セット,ホットコーヒー,NaN,NaN
2,2024/06/28 13:37:40,7,ケーキ４,ケーキ単品を注文しました,NaN,NaN
3,2024/06/28 13:49:01,7,ケーキ１セット,ホットティー,NaN,NaN
4,2024/06/28 15:04:39,10,ケーキ４セット,リンゴジュース,NaN,NaN


In [8]:
#会計実施システム（会計済入力用）　
def kaikei(number):
    reload()
    reload()
    df.fillna("a")
    flag=0
    numbers=["1","2","3","4","5","6","7","8","9","0"]
    def kakunin_popup():
      reload()
      reload()
      a=0
      popup = tk.Toplevel()
      popup.title("確認")
      popup.geometry("600x600")
      popup.configure(bg='#7fffd4')
      label = ttk.Label(popup, text="座席番号"+str(number)+
                        "番、ご注文内容は"+df.iloc[i,2]+df.iloc[i,3]+
                        ",お会計は"+str(df.iloc[i,4])+"円で間違えないですか？",
                          background='#7fffd4', foreground='#333333')
      label.pack(pady=20)
      button_frame = ttk.Frame(popup)
      button_frame.pack(pady=20)
      def yes():
        ws.update_cell(i+2,6,"会計済")
        popup.destroy()
      yes_button = ttk.Button(button_frame, text="はい", command=yes, style='MyButton.TButton', width=10)
      yes_button.pack(side="left", padx=10)
      yes_button = ttk.Button(button_frame, text="いいえ", command=popup.destroy, style='MyButton.TButton', width=10)
      yes_button.pack(side="right", padx=10)
      popup.wait_window()
      return(a)
    for i in number:
      if i not in numbers:
         flag=1
    if flag==0:
       number=int(number)
       for i in range(len(df)):
         if df.iloc[i,1]==number and not df.iloc[i,6] =="会計済":
            kakunin_popup()
            

    else:
      root = tk.Tk()
      root.withdraw()
      messagebox.showerror("エラー", "半角数字で入力してください")

In [9]:
def show_kaikei_popup():
    reload()
    reload()
    popup = tk.Toplevel()
    popup.title("入力ポップアップ")
    
    # ポップアップの大きさを変更
    popup.geometry("400x200")
    
    # ポップアップの背景色を変更
    popup.configure(bg='#7fffd4')
    
    # テキストボックスを作成
    entry = ttk.Entry(popup, width=30)
    entry.pack(pady=20)
    
    # ボタンのスタイルを変更
    style = ttk.Style()
    style.configure('MyButton.TButton', background='#7fffd4', font=('Arial', 14), padding=10)
    
    # ボタンの配置を変更
    button_frame = ttk.Frame(popup)
    button_frame.pack(pady=20)
    
    def get_text():
        text = entry.get()
        kaikei(text)
        popup.destroy()
    
    submit_button = ttk.Button(button_frame, text="送信", command=get_text, style='MyButton.TButton', width=10)
    submit_button.pack(side="right", padx=10)
    
    popup.mainloop()

In [10]:
def show_popup():
    popup = tk.Toplevel()
    popup.title("ハルモニア会計システム")
    
    # ポップアップの大きさを変更
    popup.geometry("600x600")
    
    # ポップアップの背景色を変更
    popup.configure(bg='#7fffd4')
    
    # ポップアップの内容を設定
    label = ttk.Label(popup, text="ハルモニア会計システム", background='#7fffd4', foreground='#333333')
    label.pack(pady=20)

    button_frame = ttk.Frame(popup)
    button_frame.pack(pady=50)
    
    # ボタンのスタイルを変更
    style = ttk.Style()
    style.configure('MyButton.TButton', background='#7fffd4', font=('Arial', 14), padding=10)
    
    # ボタンの大きさを変更
    count_button = ttk.Button(button_frame, text="会計更新", command=money_count, style='MyButton.TButton', width=20)
    count_button.pack(pady=20)
    kaikei_button = ttk.Button(button_frame, text="会計", command=show_kaikei_popup, style='MyButton.TButton', width=20)
    kaikei_button.pack(pady=20)    
    popup.mainloop()

In [11]:
show_popup()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\obasora\AppData\Local\Temp\ipykernel_12316\29945846.py", line 33, in money_count
    show_reload_done()
  File "C:\Users\obasora\AppData\Local\Temp\ipykernel_12316\29945846.py", line 31, in show_reload_done
    popup.mainloop()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1505, in mainloop
    self.tk.mainloop(n)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1963, in __call__
    def __call__(self, *args):
    
KeyboardInterrupt
